# Energy Cost Optimization Tool

This tool leverages data from the U.S. Energy Information Administration (EIA) API along with insights generated by a Large Language Model (LLM) API. The goal is to analyze and optimize energy costs using real-time data and advanced language processing for actionable insights.

In [ ]:
import sys
import os
import logging
import traceback
from IPython.display import clear_output  # Import to clear output

# Set logging level to debug to see more details
logging.basicConfig(level=logging.DEBUG)

# Define a list of necessary packages
required_packages = [
    "ipywidgets",
    "requests",
    "pandas",
    "python-dotenv"  # Used for handling environment variables in the .env file
]

if 'google.colab' in sys.modules:
    try:
        # Step 1: Navigate to root to reset the directory
        %cd /content

        # Step 2: Set up the repository path to a temporary directory in /content
        repo_parent_path = '/content/temporary_workspace'
        repo_path = os.path.join(repo_parent_path, 'Energy-Cost-Optimization-Tool')
        
        # Create the temporary directory if it doesn't exist
        if not os.path.exists(repo_parent_path):
            os.makedirs(repo_parent_path)
            logging.info(f"Created new parent directory: {repo_parent_path}")

        # Step 3: Check if the repository already exists and remove it to avoid conflicts
        if os.path.exists(repo_path):
            logging.info("Removing old version of the repository.")
            # Use %rm for better compatibility with Colab
            %rm -rf {repo_path}

        # Step 4: Clone the repository into the new directory
        logging.info("Cloning the repository.")
        !git clone https://github.com/hansnery/Energy-Cost-Optimization-Tool.git {repo_path}
        if not os.path.exists(repo_path):
            raise Exception("Failed to clone the repository.")
        
        %cd {repo_path}
        logging.info("Repository cloned successfully.")

        # Step 5: Install the necessary packages
        logging.info("Installing required dependencies.")
        for package in required_packages:
            %pip install {package}
        logging.info("Dependencies installed successfully.")

        # Step 6: Clear all the previous outputs
        clear_output(wait=True)

        # Reload the environment to avoid any issues with newly installed packages
        %load_ext autoreload
        %autoreload 2

        # Step 7: Import necessary modules from the cloned repository
        logging.info("Importing interface module.")
        from interface import EnergyCostOptimizationInterface
        logging.info("Imported interface module successfully.")

        # Step 8: Initialize and display the interface
        logging.info("Initializing the Energy Cost Optimization Tool interface.")
        EnergyCostOptimizationInterface()

    except Exception as e:
        logging.error("An error occurred during setup.")
        traceback.print_exc()
        print(f"Error: {e}")

else:
    print("This script is intended to run in Google Colab.")
    from interface import EnergyCostOptimizationInterface
    EnergyCostOptimizationInterface()
